## 数値型への変換

- Pythonでは数値型としてintとfloatが提供されています

In [2]:
import pandas as pd

#データ型の確認
type(40000/3)

#整数型への変換
int(40000/3)

#浮動小数点型への変換
float(40000/3)

df=pd.DataFrame({'value': [40000/3]})

#データ型の確認
df.dtypes

#整数型へ変換
df['value'].astype('int8')
df['value'].astype('int16')
df['value'].astype('int32')
df['value'].astype('int64')

#浮動小数点型へ変換
df['value'].astype('float16')
df['value'].astype('float32')
df['value'].astype('float64')
df['value'].astype('float128')

#下記のようにpythonのデータ型を指定できる
df['value'].astype(int)
df['value'].astype(float)

0    13333.333333
Name: value, dtype: float64

## 対数化による非線形な変化

- 対象のデータセットはホテルの予約レコードです。予約テーブルのtotal_priceを1000で割ってから、底１０で対数化しましょう

In [16]:
#pandasとnumpyをimportする
import numpy as np
import pandas as pd

#Pandasのread_csvを利用して、customer.csvファイルをDataFrameとして読み込み
#encodingで読み込みファイルの文字コードを設定

reserve_tb = pd.read_csv('data/reserve.csv', encoding='UTF-8')
reserve_tb.head()

,reserve_id,hotel_id,customer_id,reserve_datetime,checkin_date,checkin_time,checkout_date,people_num,total_price
0,r1,h_75,c_1,2016-03-06 13:09:42,2016-03-26,10:00:00,2016-03-29,4,97200
1,r2,h_219,c_1,2016-07-16 23:39:55,2016-07-20,11:30:00,2016-07-21,2,20600
2,r3,h_179,c_1,2016-09-24 10:03:17,2016-10-19,09:00:00,2016-10-22,2,33600
3,r4,h_214,c_1,2017-03-08 03:20:10,2017-03-29,11:00:00,2017-03-30,4,194400
4,r5,h_16,c_1,2017-09-05 19:50:37,2017-09-22,10:30:00,2017-09-23,3,68100


In [17]:
reserve_tb['total_price_log '] = reserve_tb['total_price'].apply(lambda x: np.log(x/1000 +1))
reserve_tb.head()

,reserve_id,hotel_id,customer_id,reserve_datetime,checkin_date,checkin_time,checkout_date,people_num,total_price,total_price_log
0,r1,h_75,c_1,2016-03-06 13:09:42,2016-03-26,10:00:00,2016-03-29,4,97200,4.587006
1,r2,h_219,c_1,2016-07-16 23:39:55,2016-07-20,11:30:00,2016-07-21,2,20600,3.072693
2,r3,h_179,c_1,2016-09-24 10:03:17,2016-10-19,09:00:00,2016-10-22,2,33600,3.543854
3,r4,h_214,c_1,2017-03-08 03:20:10,2017-03-29,11:00:00,2017-03-30,4,194400,5.275049
4,r5,h_16,c_1,2017-09-05 19:50:37,2017-09-22,10:30:00,2017-09-23,3,68100,4.235555


## カテゴリ化による非線形な変化

- 複雑な変化を線形モデルで表現するための前処理としてカテゴリ化という方法があります
- カテゴリ化とは数値をカテゴリ値、つまり多数のフラグ値(TRUE/FALSEしか取らない値に変換する)
- 例えば年齢を0-9才、10-19才・・・のフラグみたいな形で与える。
- さらにそれぞれのフラグに対して係数を振り分けてやることで非連続の変化を扱える。
- 機械学習モデルではカテゴリ数を増やすと学習データ量が増えてしまうので、傾向を学ぶのに問題ないカテゴリ数に抑える必要がある
- カテゴリ数を節約する方法としては例えば0-9才、10-19才、20才以上というフラグを立てた時には20才以上というフラグは必要ない
- 対象のデータセットはホテルの予約レコードです。顧客テーブルの年齢を10刻みのカテゴリ型として追加しましょう

In [18]:
#pandasとnumpyをimportする
import numpy as np
import pandas as pd

#Pandasのread_csvを利用して、customer.csvファイルをDataFrameとして読み込み
#encodingで読み込みファイルの文字コードを設定

customer_tb = pd.read_csv('data/customer.csv', encoding='UTF-8')
customer_tb.head()

,customer_id,age,sex,home_latitude,home_longitude
0,c_1,41,man,35.092193,136.512347
1,c_2,38,man,35.325076,139.410551
2,c_3,49,woman,35.120543,136.511179
3,c_4,43,man,43.034868,141.240314
4,c_5,31,man,35.102661,136.523797


In [19]:
customer_tb['age_rank'] = (np.floor(customer_tb['age']/10)*10).astype('category')
customer_tb.head()

,customer_id,age,sex,home_latitude,home_longitude,age_rank
0,c_1,41,man,35.092193,136.512347,40.0
1,c_2,38,man,35.325076,139.410551,30.0
2,c_3,49,woman,35.120543,136.511179,40.0
3,c_4,43,man,43.034868,141.240314,40.0
4,c_5,31,man,35.102661,136.523797,30.0


## 正規化

- 機械学習手法を利用する際に、列ごとに取りうる範囲が大きく異なると問題が発生する場合がある。
- その場合は正規化しましょう。よく取りうる正規化の手法は以下
- 1. 平均0、分散1に変換する正規化
- →(入力値-平均値)/入力値の標準偏差
- 2. 最小値0、最大値1に変換する正規化
- →(入力値-最小値)/(最大値-最小値)
- 対象のデータセットはホテルの予約レコードです。予約テーブルの予約人数(people_num)と合計金額(total_price)を
- 平均0、分散1の分布に変換し、正規化しましょう

In [20]:
#Pandasのread_csvを利用して、customer.csvファイルをDataFrameとして読み込み
#encodingで読み込みファイルの文字コードを設定

reserve_tb = pd.read_csv('data/reserve.csv', encoding='UTF-8')

#PythonではsklearnのStandardScalerクラスを使うことで平均0、分散1に変換する正規化を実現できる
from sklearn.preprocessing import StandardScaler

#小数点以下を扱えるようにするためにfloat型へ変換
reserve_tb['people_num']= reserve_tb['people_num'].astype(float)

#正規化を行うオブジェクトを生成
#最小値0、最大値1に変換する正規化はMinMaxScalerを使う
ss=StandardScaler()

#fit_transorm関数はfit関数(正規化するための前準備の計算)とtransform関数(準備された情報から正規化の変換処理を行う)の両方を行う
result = ss.fit_transform(reserve_tb[['people_num', 'total_price']])

reserve_tb['people_num_normalized']=[x[0] for x in result]
reserve_tb['total_price_normalized']=[x[1] for x in result]

reserve_tb.head()

,reserve_id,hotel_id,customer_id,reserve_datetime,checkin_date,checkin_time,checkout_date,people_num,total_price,people_num_normalized,total_price_normalized
0,r1,h_75,c_1,2016-03-06 13:09:42,2016-03-26,10:00:00,2016-03-29,4.0,97200,1.300709,-0.053194
1,r2,h_219,c_1,2016-07-16 23:39:55,2016-07-20,11:30:00,2016-07-21,2.0,20600,-0.483753,-0.747822
2,r3,h_179,c_1,2016-09-24 10:03:17,2016-10-19,09:00:00,2016-10-22,2.0,33600,-0.483753,-0.629935
3,r4,h_214,c_1,2017-03-08 03:20:10,2017-03-29,11:00:00,2017-03-30,4.0,194400,1.300709,0.828240
4,r5,h_16,c_1,2017-09-05 19:50:37,2017-09-22,10:30:00,2017-09-23,3.0,68100,0.408478,-0.317080


## 外れ値の除去

- 一般的によく使われる正規分布を前提とした外れ値除去の方法
- 予約テーブルの予約合計金額(total_price)において、平均値から標準偏差値の3倍以内の値に収まる
- 予約レコードのみに絞る

In [30]:
#pandasとnumpyをimportする
import numpy as np
import pandas as pd

#Pandasのread_csvを利用して、customer.csvファイルをDataFrameとして読み込み
#encodingで読み込みファイルの文字コードを設定

reserve_tb = pd.read_csv('data/reserve.csv', encoding='UTF-8')

reserve_tb2 = reserve_tb[(abs(reserve_tb['total_price']-np.mean(reserve_tb['total_price']))
                         /np.std(reserve_tb['total_price']) <= 3)].reset_index()

reserve_tb2.describe()

,index,people_num,total_price
count,3932.000000,3932.000000,3932.000000
mean,2010.104527,2.512716,91814.165819
std,1163.646983,1.116738,83620.043510
min,0.000000,1.000000,3500.000000
25%,1004.750000,2.000000,31800.000000
50%,2010.500000,3.000000,63250.000000
75%,3012.250000,4.000000,123000.000000
max,4029.000000,4.000000,420600.000000


## 主成分分析による次元圧縮

- 次元圧縮の指標としては寄与率がある。これは元のデータの何割を説明できているかを表した値
- 寄与率は指定した主成分軸1つでデータの変動の何割を説明感冒なのかを表したもの
- この値が高いほど、次元圧縮した際の情報損失が少ないということを意味している
- (情報損失とはデータの点から新たな軸に対しての距離のことをいう)
- 通常は寄与率の合計値(累積寄与率)を目安にしており、90%以上になる次元数を採用することが多い。
- 主成分分析自体は機械学習モデル精度の向上というより、データ可視化を容易にする側面が強い
- 製造レコードを使ってlengthとthicknessを一次元に圧縮しましょう

In [2]:
import numpy as np
import pandas as pd

production_tb = pd.read_csv('data/production.csv', encoding='UTF-8')

#PCA読み込み
from sklearn.decomposition import PCA

#n_componentsに、主成分分析で変換後の次元数を設定
pca=PCA(n_components=2)

#主成分分析を実行
#pcaに主成分分析の変換パラメータが保存され、返り値に主成分分析後の値が返される
pca_values = pca.fit_transform(production_tb[['length', 'thickness']])

#累積寄与率と寄与率の確認
print('累積寄与率:{0}'.format(sum(pca.explained_variance_ratio_)))
print('各次元の寄与率:{0}'.format(pca.explained_variance_ratio_))

#predict関数を利用し、同じ次元圧縮処理を実行
pca_newvalues = pca.transform(production_tb[['length', 'thickness']])

pd.DataFrame(pca_newvalues).head()

累積寄与率:1.0
各次元の寄与率:[0.97897794 0.02102206]


,0,1
0,76.968382,-13.389069
1,-112.114693,-8.248848
2,-76.199434,11.190271
3,-23.341625,0.838485
4,46.933896,-5.064103


## 数値の補完

- センサーデータの欠損値が多い
- MCAR(missing completely at random)：偶然に起きている完全なランダムな欠損
- MAR(missing at random)：欠損した項目データとは関係なく、他の項目データに依存した欠損
- MNAR(missing not at random):欠損した項目データに依存した欠損です。
- MCAR、MARについては多重代入法、最尤法を利用することが一般的

## 多重代入法の補完
- 欠損レコードの削除についてthicknessに欠損が存在する製造レコードでthicknessが欠損しているレコードを削除しましょう

In [3]:
import numpy as np
from preprocess.load_data.data_loader import load_production_missing_num
production_miss_num = load_production_missing_num()

#replace関数によってNoneをnanに変換
#(Noneを指定する際には文字列として指定する必要がある)
production_miss_num.replace('None', np.nan, inplace=True)

#dropna関数によって、thicknessにnanを含むレコードを削除。subsetを指定しない場合は全ての列が対象になる
production_miss_num.dropna(subset=['thickness'],inplace=True)
production_miss_num.reset_index().head()

,index,type,length,thickness,fault_flg
0,0,E,274.027383,40.24113135955541,False
1,1,D,86.319269,16.906714630016268,False
2,2,E,123.940388,1.0184619943950775,False
3,3,B,175.554886,16.41492419553766,False
4,4,B,244.934740,29.061080805480326,False


## 定数補完
- 欠損レコードの削除についてthicknessに欠損が存在する製造レコードで欠損しているthicknessを1で補完しましょう

In [4]:
import numpy as np
from preprocess.load_data.data_loader import load_production_missing_num
production_miss_num = load_production_missing_num()

#replace関数によってNoneをnanに変換
production_miss_num.replace('None', np.nan, inplace=True)

#fillna関数によって、thicknessの欠損値を1で補完
production_miss_num['thickness'].fillna(1, inplace=True)

## 平均値補完
- 欠損レコードの削除についてthicknessに欠損が存在する製造レコードで欠損しているthicknessを欠損していないthicknessの平均で補完しましょう

In [5]:
import numpy as np
from preprocess.load_data.data_loader import load_production_missing_num
production_miss_num = load_production_missing_num()

#replace関数によってNoneをnanに変換
production_miss_num.replace('None', np.nan, inplace=True)

#thicknessを数値型に変換(Noneが混ざっているため数値型になっていない)
production_miss_num['thickness']=production_miss_num['thickness'].astype('float64')

#thicknessの平均値を計算
thickness_mean=production_miss_num['thickness'].mean()

#thicknessの欠損値をthicknessの平均値で補完
production_miss_num['thickness'].fillna(thickness_mean, inplace=True)

## PMMによる多重代入(あとで作る)
- 欠損レコードの削除についてthicknessに欠損が存在する製造レコードで欠損しているthicknessを多重代入法で補完しましょう
- いくつかある多重代入法の中でPMM(Predictive Mean Matching)を利用しましょう
- 1. 欠測データを除いたデータから欠損データを予測する回帰モデルを構築
- 2. 構築した回帰モデルの係数と誤差分散の分布を計算
- 3. 係数と誤差分散の分布から新たな係数と誤差分散の値を生成
- 4. 3.で生成した係数と誤差分散の値に従った回帰モデルから予測値を計算
- 5. 欠損していない観測データの中から予測値に最も近いデータを補完値として採用
- 6. データを補完して、新たな構築した回帰モデルの係数と誤差分散の分布を計算、3に戻る

In [13]:
import numpy as np
import pandas as pd
from preprocess.load_data.data_loader import load_production_missing_num
production_miss_num = load_production_missing_num()

#sklearnのIterativeImputerを使用する

from fancyimpute import IterativeImputer
MICE_imputer = IterativeImputer()
diabetes_MICE = diabetes.copy(deep=True)
diabetes_MICE.iloc[:, :] = MICE_imputer.fit_transform(diabetes_MICE)

#replace関数によってNoneをnanに変換
production_miss_num.replace('None', np.nan, inplace=True)

#MICE関数を利用するためにデータ型に変換
production_miss_num['thickness']=production_miss_num['thickness'].astype('float64')
production_miss_num['type']=production_miss_num['type'].astype('category')
production_miss_num['fault_flg']=production_miss_num['fault_flg'].astype('category')

#ダミー変数化(第9章 カテゴリ型」で詳しく解説)
production_dummy_flg = pd.get_dummies(production_miss_num[['type', 'fault_flg']], drop_first=True)

#mice関数にPMMを指定して、多重代入法を実施
#n_imputationsは取得するデータセットの数
#n_burn_inは値を取得する前に試行する回数
mice =MICE(n_imputations=10, n_burn_in=50, impute_type='pmm')

diabetes_MICE.iloc[:, :] = MICE_imputer.fit_transform(diabetes_MICE)

#処理内部でTensorFlowを利用
production_mice = mice.multiple_imputations(
  #数値の列とダミー変数を連結
  pd.concat([production_miss_num[['length', 'thickness']],
            production_dummy_flg], axis=1)
)

#下記に補完する値が格納されている
production_mice[0]

NotFittedError: This IterativeImputer instance is not fitted yet. Call 'fit' with appropriate arguments before using this method.

## カテゴリ型への変換

- カテゴリ型は取りうる値の種類が決まっている値です(例えば居住都道府県の列(47都道府県名)、会員のステータス(true/false)
- 数値をカテゴリ化した時にはカテゴリ値間の関係性のデータは表現されない
- 一見すると、カテゴリ型は文字型や数値型のデータと大差ありません
-　ほとんどのプログラムは文字列や数値型としてデータを読み込みます
- カテゴリ型としてデータを扱うにはカテゴリ型のデータに変換する必要がある

- ホテルの予約コードを用いて顧客テーブルの性別(sex)をブール型とカテゴリ型に変換してみましょう

In [16]:
# pandasとnumpyをimportする
import numpy as np
import pandas as pd

# Pandasのread_csvを利用して、customer.csvファイルをDataFrameとして読み込み
# encodingで読み込みファイルの文字コードを設定

customer_tb = pd.read_csv('data/customer.csv', encoding='UTF-8')
customer_tb.head()

#  sexがmanの時にTRUEとするブール型を追加
#このコードは、astype関数を利用しなくてもブール型に変換
customer_tb[['sex_is_man']] = (customer_tb[['sex']] == 'man').astype('bool')

# sexをカテゴリ型に変換
customer_tb['sex_c'] = pd.Categorical(customer_tb['sex'], categories=['man', 'woman'])

# astype関数でも変換可能
# customer_tb['sex_c'] = customer_tb['sex_c'].astype('category')

#インデックスデータはcodesに格納されている
customer_tb['sex_c'].cat.codes

#マスターデータはcategoriesに格納されている
customer_tb['sex_c'].cat.categories

customer_tb.head()

,customer_id,age,sex,home_latitude,home_longitude,sex_is_man,sex_c
0,c_1,41,man,35.092193,136.512347,True,man
1,c_2,38,man,35.325076,139.410551,True,man
2,c_3,49,woman,35.120543,136.511179,False,woman
3,c_4,43,man,43.034868,141.240314,True,man
4,c_5,31,man,35.102661,136.523797,True,man


## ダミー変数化
- 一部のメソッドがカテゴリ型に対応していない時に、フラグを立てて対応する必要がある
- カテゴリー値に対してそれぞれフラグを立てることを「ダミー変数化」という
- PythonではPandasのライブラリのget_dummies関数によってダミー変数化を実現できます

In [21]:
#ダミー変数化する前にカテゴリ型に変換
customer_tb['sex'] = pd.Categorical(customer_tb['sex'])

#get_dummies関数によってsexダミー変数化
#drop_firstをFalseにすると、カテゴリ値の全種類の値のダミーフラグを生成
dummy_vars = pd.get_dummies(customer_tb['sex'], drop_first=False)
dummy_vars.head()

,man,woman
0,1,0
1,1,0
2,0,1
3,1,0
4,1,0


## カテゴリ値の集約
- データ数の少ないカテゴリ値が存在すると過学習する可能性があるので、
- 他のカテゴリ値と集約することがある(例えば10才未満と10代をまとめるとか)


### 問題
- ホテルの予約コードを用いて顧客テーブルの年齢を10才区切りでカテゴリ型に変換し、
- さらに60才以上の場合は"60才以上というカテゴリに変換

In [26]:
# pandasとnumpyをimportする
import numpy as np
import pandas as pd

# Pandasのread_csvを利用して、customer.csvファイルをDataFrameとして読み込み
# encodingで読み込みファイルの文字コードを設定

customer_tb = pd.read_csv('data/customer.csv', encoding='UTF-8')

#pd.Categoricalによって、category型に変換
customer_tb['age_rank'] = pd.Categorical(np.floor(customer_tb['age']/10)*10)

#マスタデータに'60以上'を追加
customer_tb['age_rank'].cat.add_categories(['60以上'], inplace=True)

#集約するデータを書き換え
#category型は、=または！=の判定のみ可能なので、isin関数を利用
customer_tb.loc[customer_tb['age_rank'].isin([60.0, 70.0, 80.0]), 'age_rank'] = '60以上'

#利用されていないマスタデータを削除
customer_tb['age_rank'].cat.remove_unused_categories(inplace=True)

customer_tb.head(10)

,customer_id,age,sex,home_latitude,home_longitude,age_rank
0,c_1,41,man,35.092193,136.512347,40
1,c_2,38,man,35.325076,139.410551,30
2,c_3,49,woman,35.120543,136.511179,40
3,c_4,43,man,43.034868,141.240314,40
4,c_5,31,man,35.102661,136.523797,30
5,c_6,52,man,34.440768,135.390487,50
6,c_7,50,man,43.015758,141.231321,50
7,c_8,65,woman,38.201268,140.465961,60以上
8,c_9,36,woman,33.322800,130.330689,30
9,c_10,34,woman,34.290414,132.302601,30


## カテゴリ値の組み合わせ
- カテゴリ値を組み合わせることでカテゴリ値の種類を増やす場合もあります
- 男性20代、女性50代と行った新たなカテゴリ値に拡張します。これにより、
- 男女によって同じ年代でも大きく傾向が異なっていたとしても、カテゴリ値によって
- 非線形な変化を表現できるが、組み合わせによってはダミー変数の量が膨大になる
- 可能性があるので注意(データ容量という意味で)

### 問題
- ホテルの予約コードを用いて顧客テーブルの性別と年齢を10才区切りで
- カテゴリ値を組み合わせて性別/年代のカテゴリ値を生成しましょう

In [29]:
# pandasとnumpyをimportする
import numpy as np
import pandas as pd

# Pandasのread_csvを利用して、customer.csvファイルをDataFrameとして読み込み
# encodingで読み込みファイルの文字コードを設定

customer_tb = pd.read_csv('data/customer.csv', encoding='UTF-8')

customer_tb['sex_and_age']=pd.Categorical(
    #連結する列を抽出
    customer_tb[['sex', 'age']]
    
    #lambda関数内でsexと10代区切りのageを_を挟んで文字列として連結
    .apply(lambda x: '{}_{}'.format(x[0], np.floor(x[1]/10)*10), axis=1)
)
customer_tb.head()

,customer_id,age,sex,home_latitude,home_longitude,sex_and_age
0,c_1,41,man,35.092193,136.512347,man_40.0
1,c_2,38,man,35.325076,139.410551,man_30.0
2,c_3,49,woman,35.120543,136.511179,woman_40.0
3,c_4,43,man,43.034868,141.240314,man_40.0
4,c_5,31,man,35.102661,136.523797,man_30.0


## カテゴリ型の補完
- カテゴリ型のデータが欠損している場合は数値欠損と同様にどのような欠損かを
- 確認した後に適切な対処を選択する必要があります

### KNNによる補完
- fault_flgに欠損が存在する製造レコードがある。欠損していないデータを用いた
- 予測結果から、欠損しているfault_flgを補完する。予測にはKNNを利用。

In [41]:
import numpy as np
import pandas as pd

production_missc_tb = pd.read_csv('data/production_missing_category.csv', encoding='UTF-8')

#KNeighborsClassifierをsklearnから読み込み
from sklearn.neighbors import KNeighborsClassifier

#replace関数によって、Noneをnanに変換
production_missc_tb.replace('None', np.nan, inplace=True)

#欠損していないデータの抽出
train = production_missc_tb.dropna(subset=['type'], inplace=False)

#欠損しているデータの抽出
test = production_missc_tb.loc[production_missc_tb.index.difference(train.index), :]

#knnモデル生成、n_neighborsはknnのkパラメータ
kn = KNeighborsClassifier(n_neighbors=3)

#knnモデル学習
kn.fit(train[['length', 'thickness']], train['type'])

#knnモデルによって予測値を計算し、typeを補完
test['type'] = kn.predict(test[['length', 'thickness']])
